In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib
import seaborn 
import folium
import numpy as np
from folium.plugins import HeatMap
from folium import plugins
import webbrowser
from tqdm import tqdm

In [2]:
latlong = pd.read_csv('ID_latlong_m.csv')
latlong.describe()

,longitude,latitude,OBJECTID,Shape_Leng,Shape_Area,LocationID
count,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000
mean,-73.931246,40.725965,132.000000,0.094269,0.000402,131.984791
std,0.095904,0.085432,76.065761,0.054594,0.000482,76.073787
min,-74.229547,40.527298,1.000000,0.014306,0.000006,1.000000
25%,-73.986156,40.672690,66.500000,0.054722,0.000133,66.500000
50%,-73.938311,40.727347,132.000000,0.084341,0.000268,132.000000
75%,-73.870456,40.779549,197.500000,0.119488,0.000480,197.500000
max,-73.709071,40.899860,263.000000,0.433470,0.004866,263.000000


In [3]:
zone=pd.read_csv('../Project_data/taxi+_zone_lookup.csv')

In [4]:
month7=pd.read_csv('../Project_data/yellow_tripdata_2019-07.csv')
month7.head()

/root/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2.0,2019-07-01 00:51:04,2019-07-01 00:51:33,1.0,0.00,1.0,N,193,193,1.0,2.5,0.5,0.5,1.14,0.00,0.3,4.94,0.0
1,2.0,2019-07-01 00:46:04,2019-07-01 01:05:46,1.0,4.16,1.0,N,234,25,2.0,16.5,0.5,0.5,0.00,0.00,0.3,20.30,2.5
2,1.0,2019-07-01 00:25:09,2019-07-01 01:00:56,1.0,18.80,2.0,N,132,42,1.0,52.0,0.0,0.5,11.75,6.12,0.3,70.67,0.0
3,2.0,2019-07-01 00:33:32,2019-07-01 01:15:27,1.0,18.46,2.0,N,132,142,1.0,52.0,0.0,0.5,11.06,0.00,0.3,66.36,2.5
4,1.0,2019-07-01 00:00:55,2019-07-01 00:13:05,0.0,1.70,1.0,N,107,114,1.0,9.5,3.0,0.5,2.00,0.00,0.3,15.30,2.5


In [6]:
# month7.groupby(['PULocationID','DOLocationID']).count()

In [5]:
# Convert time str into datetime
month7['tpep_pickup_datetime'] = pd.to_datetime(month7['tpep_pickup_datetime'])
(month7['tpep_pickup_datetime'][1] - month7['tpep_pickup_datetime'][2]).total_seconds()

1255.0

In [6]:
float(latlong.longitude[262])

-73.95120799

In [7]:
month7.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,6.276460e+06,6.276460e+06,6.310419e+06,6.276460e+06,6.310419e+06,6.310419e+06,6.276460e+06,6.310419e+06,6.310419e+06,6.310419e+06,6.310419e+06,6.310419e+06,6.310419e+06,6.310419e+06,6.310419e+06
mean,1.647390e+00,1.572045e+00,3.110132e+00,1.061235e+00,1.616448e+02,1.597920e+02,1.305580e+00,1.361008e+01,1.163916e+00,4.936708e-01,2.183854e+00,4.055411e-01,2.984449e-01,1.961230e+01,2.260082e+00
std,4.881620e-01,1.214311e+00,4.065832e+00,7.495861e-01,6.566174e+01,7.009963e+01,4.877842e-01,1.311566e+01,1.277882e+00,6.457269e-02,2.875575e+00,1.890058e+00,2.903123e-02,1.581046e+01,7.495733e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-4.500000e+02,-7.000000e+00,-5.000000e-01,-2.210000e+02,-3.324000e+01,-3.000000e-01,-4.508000e+02,-2.500000e+00
25%,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.140000e+02,1.070000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.130000e+01,2.500000e+00
50%,2.000000e+00,1.000000e+00,1.680000e+00,1.000000e+00,1.610000e+02,1.620000e+02,1.000000e+00,9.500000e+00,5.000000e-01,5.000000e-01,1.860000e+00,0.000000e+00,3.000000e-01,1.476000e+01,2.500000e+00
75%,2.000000e+00,2.000000e+00,3.190000e+00,1.000000e+00,2.310000e+02,2.320000e+02,2.000000e+00,1.500000e+01,2.500000e+00,5.000000e-01,2.950000e+00,0.000000e+00,3.000000e-01,2.130000e+01,2.500000e+00
max,4.000000e+00,9.000000e+00,3.115600e+02,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,6.666650e+03,1.850000e+01,6.800000e+00,4.195600e+02,1.559120e+03,3.000000e-01,6.667450e+03,2.750000e+00


In [8]:
# data cleaning 264 265
C = list(month7[month7['PULocationID']==264].index)
D = list(month7[month7['PULocationID']==265].index)
E = list(month7[month7['PULocationID']!=264][month7['PULocationID']!=265][month7['DOLocationID']==265].index)
F = list(month7[month7['PULocationID']!=264][month7['PULocationID']!=265][month7['DOLocationID']==264].index)
print(len(C),len(D),len(E),len(F))
month7=month7.drop(labels=C,axis=0)
month7=month7.drop(labels=D,axis=0)

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


49261 4149 15807 16538


In [9]:
month7=month7.drop(labels=E,axis=0)
month7=month7.drop(labels=F,axis=0)
month7.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,6.191052e+06,6.191052e+06,6.224664e+06,6.191052e+06,6.224664e+06,6.224664e+06,6.191052e+06,6.224664e+06,6.224664e+06,6.224664e+06,6.224664e+06,6.224664e+06,6.224664e+06,6.224664e+06,6.224664e+06
mean,1.647555e+00,1.571906e+00,3.080700e+00,1.046936e+00,1.608536e+02,1.585522e+02,1.304141e+00,1.335765e+01,1.169878e+00,4.948437e-01,2.159481e+00,3.908710e-01,2.987053e-01,1.933934e+01,2.276974e+00
std,4.882232e-01,1.214528e+00,3.937124e+00,6.188549e-01,6.527500e+01,6.957516e+01,4.863872e-01,1.202895e+01,1.278952e+00,5.954163e-02,2.732500e+00,1.710295e+00,2.731208e-02,1.466323e+01,7.262677e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-4.000000e+02,-4.500000e+00,-5.000000e-01,-2.210000e+02,-3.324000e+01,-3.000000e-01,-4.033000e+02,-2.500000e+00
25%,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.140000e+02,1.070000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.130000e+01,2.500000e+00
50%,2.000000e+00,1.000000e+00,1.690000e+00,1.000000e+00,1.610000e+02,1.610000e+02,1.000000e+00,9.500000e+00,5.000000e-01,5.000000e-01,1.860000e+00,0.000000e+00,3.000000e-01,1.476000e+01,2.500000e+00
75%,2.000000e+00,2.000000e+00,3.170000e+00,1.000000e+00,2.310000e+02,2.310000e+02,2.000000e+00,1.500000e+01,2.500000e+00,5.000000e-01,2.950000e+00,0.000000e+00,3.000000e-01,2.100000e+01,2.500000e+00
max,4.000000e+00,9.000000e+00,3.075000e+02,9.900000e+01,2.630000e+02,2.630000e+02,4.000000e+00,6.666650e+03,1.850000e+01,6.800000e+00,4.195600e+02,6.181200e+02,3.000000e-01,6.667450e+03,2.750000e+00


In [10]:
i_month7 = month7.index.tolist()
len(i_month7)

6224664

In [4]:
# Add latlong to month dataset 
# additional columns PUlatlong DOlatlong
'''
PUlat,PUlong, DOlat,DOlong =[],[],[],[]
for i in i_month7:
    PUID = month7.PULocationID[i]-1
    DOID = month7.DOLocationID[i]-1
    PUlong.append(float(latlong.longitude[PUID]))
    PUlat.append(float(latlong.latitude[PUID]))
    DOlong.append(float(latlong.longitude[DOID]))
    DOlat.append(float(latlong.latitude[PUID]))
''

'\nPUlat,PUlong, DOlat,DOlong =[],[],[],[]\nfor i in i_month7:\n    PUID = month7.PULocationID[i]-1\n    DOID = month7.DOLocationID[i]-1\n    PUlong.append(float(latlong.longitude[PUID]))\n    PUlat.append(float(latlong.latitude[PUID]))\n    DOlong.append(float(latlong.longitude[DOID]))\n    DOlat.append(float(latlong.latitude[PUID]))\n'

In [ ]:
month7['PUlat'] = PUlat
month7['PUlong'] = PUlong
month7['DOlat'] = DOlat
month7['DOlong'] = DOlong
month7.head()

In [ ]:
month7.to_csv('month7+.csv')

In [12]:
month7PLUS = pd.read_csv('month7+.csv')

/root/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
def in_10(t_1,t_2):
    t = (t_1 - t_2).total_seconds()
    if t > 600: 
        return False 
    else:
        return True 
    
def in_20(t_1,t_2):
    t = (t_1 - t_2).total_seconds()
    if t > 1200: 
        return False 
    else:
        return True 

In [2]:
# willingness 
# if 50% of the single passengers are willing to share the trip
alpha = 0.5
month7PLUS['tpep_pickup_datetime'] = pd.to_datetime(month7PLUS['tpep_pickup_datetime'])
C=[]
sample7 = month7PLUS.sample(n=6000000)

In [1]:
# caution: long running time  5 hr
for ID_1 in tqdm(range(0,263)):
    for ID_2 in range(0,263): # 265
        PU_ID= list(sample7[sample7['PULocationID']==ID_1][sample7['DOLocationID']==ID_2].index)
        c_1 = 0
        for i in PU_ID:
            for j in PU_ID:
                if i < j:
                    if in_10(sample7['tpep_pickup_datetime'][i],sample7['tpep_pickup_datetime'][j]):
                        C.append(i)   

"\nfor ID_1 in tqdm(range(0,263)):\n    for ID_2 in range(0,263): # 265\n        PU_ID= list(sample7[sample7['PULocationID']==ID_1][sample7['DOLocationID']==ID_2].index)\n        c_1 = 0\n        for i in PU_ID:\n            for j in PU_ID:\n                if i < j:\n                    if in_10(sample7['tpep_pickup_datetime'][i],sample7['tpep_pickup_datetime'][j]):\n                        C.append(i)   \n"

In [ ]:
len(C)
month7_carpool = month7PLUS.iloc[C, :]
month7_carpool.describe()

In [ ]:
'''
month7PLUS['tpep_pickup_datetime'] = pd.to_datetime(month7PLUS['tpep_pickup_datetime'])
C_20=[]
for ID_1 in range(30,40):
    for ID_2 in range(30,40): # 265
        PU_ID= list(month7PLUS[month7PLUS['PULocationID']==ID_1][month7PLUS['DOLocationID']==ID_2].index)
        for i in PU_ID:
            for j in PU_ID:
                if i < j:
                    if in_20(month7PLUS['tpep_pickup_datetime'][i],month7PLUS['tpep_pickup_datetime'][j]):
                        C_20.append(i)      # index of trips reduced by carpooling
'''

In [ ]:
month7_reduced=month7PLUS.drop(labels=C,axis=0)
month7_reduced.describe()

In [ ]:
len(month7_reduced)

In [ ]:
month7_reduced.to_csv('month7_reduced.csv')

In [ ]:
def density_map(all_data,filename):
 
    lat = np.array(all_data["PUlat"][0:len(all_data)])
    lon = np.array(all_data["PUlong"][0:len(all_data)])
 
    # duration is not essential
    columns_list = all_data.columns
    if 'duration' in columns_list:
        duration = np.array(all_data["duration"][0:len(all_data)], dtype=float)
        new_data = [[lat[i], lon[i], duration[i]] for i in range(len(all_data))]
    else:
        new_data = [[lat[i], lon[i]] for i in range(len(all_data))]
 
    density_map = folium.Map(location=[all_data['PUlat'].mean(), all_data['PUlong'].mean()],
                             tiles= "Stamen Terrain",
                             zoom_start = 13,
                             attr="Mapbox attribution")
   
 
    HeatMap(new_data, radius=20).add_to(density_map)
    #blur=15, gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}
    file_path = filename
    density_map.save(file_path)     # 保存为html文件
    webbrowser.open(file_path)

    

In [ ]:
density_map(sample7,r"sample7_600.html")
sample7_reduced=sample7.drop(labels=C,axis=0)
sample7_reduced_600.to_csv('month7_reduced_600.csv')
density_map(sample7_reduced,r"sample7_reduced_600.html")

In [ ]:
#density_map(month7_carpool,r"carpool_7.html") #Trust html

In [ ]:
import geopandas as gd
import contextily as ctx
import numpy as np

In [ ]:
df=gd.read_file('../Nicolas.H/Shape/taxi_zones.shp')

In [ ]:
weight=np.array([i+100 for i in range(263)])

In [ ]:
#ax = df.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
df = df.to_crs(epsg=3857)

areas = [[]]

ax = df.plot( column = weight ,figsize=(10, 10), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax)